# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [135]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [136]:
# Load the csv file to a Pandas Dataframe and print it out
cities_file = pd.read_csv('cities_df.csv')
cities_file

,name,temperature,humidity,cloudiness,wind_speed,latitude,longitude
0,Lebedyan',2.00,93,90,7.00,53.02,39.17
1,Atlantic City,12.63,22,75,4.10,39.36,-74.42
2,Iqaluit,-25.77,62,20,6.70,63.75,-68.51
3,Punta Arenas,15.00,58,75,9.30,-53.15,-70.92
4,Dubbo,12.00,81,0,5.10,-32.25,148.62
...,...,...,...,...,...,...,...
537,Scottsbluff,0.11,80,90,7.70,41.87,-103.67
538,Yining,7.67,67,0,0.40,29.04,114.56
539,Fes,19.00,63,40,3.60,34.04,-5.00
540,Karaton,25.80,83,20,1.50,-6.34,106.10


In [137]:
# convert lat to list, convert lon to list or both to dictionary?

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [138]:
import gmaps

# Google developer API key
from config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [139]:
# # Create a list containing coordinates
# df = pd.DataFrame(cities_file, columns= ['latitude', 'longitude'])
# coordinates_list = df.values.tolist()
# print(coordinates_list)

In [140]:
# # Customize the size of the figure
# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }
# fig = gmaps.figure(layout=figure_layout)
# fig

In [141]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_file[["latitude", "longitude"]]

cities_file = cities_file.dropna()
humidity_metrics = cities_file["humidity"]

In [150]:
# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_metrics, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [143]:
# Create variables to hold the ideal conditions for our vacation spots
# Source code: https://chrisalbon.com/python/data_wrangling/pandas_selecting_rows_on_conditions/
min_temp = cities_file["temperature"] > 21.1
max_temp = cities_file["temperature"] > 26.7
wind_speed = cities_file["wind_speed"] < 10
cloudiness = cities_file["cloudiness"]==0

ideal_df = cities_file[min_temp & max_temp & wind_speed & cloudiness]

ideal_df = ideal_df.dropna()
ideal_df.head()

,name,temperature,humidity,cloudiness,wind_speed,latitude,longitude
79,Imbituba,28.00,61,0,4.60,-28.24,-48.67
85,Cachoeirinha,35.17,27,0,2.60,-29.95,-51.09
348,Araras,31.00,38,0,1.00,-22.36,-47.38
500,Tomatlán,30.67,50,0,4.11,19.93,-105.25


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [144]:
ideal_df['hotel_name'] = ""

In [145]:
ideal_df

,name,temperature,humidity,cloudiness,wind_speed,latitude,longitude,hotel_name
79,Imbituba,28.00,61,0,4.60,-28.24,-48.67,
85,Cachoeirinha,35.17,27,0,2.60,-29.95,-51.09,
348,Araras,31.00,38,0,1.00,-22.36,-47.38,
500,Tomatlán,30.67,50,0,4.11,19.93,-105.25,


In [160]:
## Use the following code to retrieve a hotel in these ideal cities

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in ideal_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        ideal_df.loc["hotel_name"] = name_address['results'][7]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [165]:
# It looks like we may not yet know our hotels!
ideal_df

,name,temperature,humidity,cloudiness,wind_speed,latitude,longitude,hotel_name
79,Imbituba,28,61,0,4.6,-28.24,-48.67,
85,Cachoeirinha,35.17,27,0,2.6,-29.95,-51.09,
348,Araras,31,38,0,1,-22.36,-47.38,
500,Tomatlán,30.67,50,0,4.11,19.93,-105.25,
hotel_name,Hospital de Primer Contacto Tomatlan,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map
# Customize the size of the figure
# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display Map